In [ ]:
#!pip install pandas numpy requests
#!pip install beautifulsoup4
#!pip install selenium webdriver-manager beautifulsoup4
#!pip install --upgrade typing_extensions
#!pip install --upgrade selenium
#!pip install --upgrade bleach tinycss2



In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import os
import io

# Google Drive API dependencies
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseUpload

import tempfile


# Get the temporary directory
tmp_dir = tempfile.gettempdir()
print(f"Temporary directory: {tmp_dir}")

# Create a temporary file inside the temp directory # Filepath for CSV
csv_filename = os.path.join(tmp_dir, "data.csv")



# Base URLs
csv_base_url = "https://stooq.pl/q/d/l/?s={}.n&i=d"
title_base_url = "https://stooq.pl/q/g/?s={}.n"




# List to store all results
all_results = []



def get_webpage_title(url):
    # Set up Chrome options for headless mode (no GUI)
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--user-data-dir=/tmp/chrome_test_data') 
    options.add_argument('--disable-dev-shm-usage')
    
    print("URL: ", url)
    
    # Automatically download and manage ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    print("URL: ", url)
    # Open the webpage URL
    driver.get(url)
    
    # Wait for the cookie acceptance pop-up to appear
    try:
        # Wait until the button is present and clickable
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "fc-cta-consent"))
        )
        accept_button.click()  # Click the accept button
        print("Cookie pop-up accepted.")
    
    except Exception as e:
        print("No cookie acceptance pop-up found or timeout occurred:", e)
    
    # Wait for the page to load fully
    time.sleep(3)  
    
    # Get the rendered HTML of the page
    page_source = driver.page_source
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Look for the <title> tag
    title_tag = soup.find('title')
    if title_tag:
        title = title_tag.get_text(strip=True)
    else:
        title = "No <title> tag found"
    
    # Close the Selenium driver
    driver.quit()
    
    return title




# Main function to process the data
def process_data(url, url_title, filename):

    # Get the title of the webpage
    nazwa = get_webpage_title(url_title)
    if nazwa == "Wyszukiwanie symbolu - Stooq" or nazwa == "No <title> tag found":
        print("⚠️ Brak symbolu w bazie Stooq")
        return None
    
    # Save the required data into an array
    result = [
        nazwa,  # Name of the file
        ]
    
    print("✅ Processed Data:", result)
    return result

In [ ]:
# Loop through each XXXX value
for xxxx in range(1000, 1010):
    csv_url = csv_base_url.format(xxxx)
    title_url = title_base_url.format(xxxx)

    print(f"Processing: {csv_url}")
    result = process_data(csv_url, "https://stooq.pl/q/g/?s=1006.n", csv_filename)

    all_results.append(result)
   # print("✅ Processed and appended Data:", all_results)
    # Delete CSV file after processing
    if os.path.exists(csv_filename):
        os.remove(csv_filename)
    
    time.sleep(2)  # Optional delay

In [7]:
# Convert results to DataFrame and save
cleaned_results = [row for row in all_results if row is not None]
final_df = pd.DataFrame(cleaned_results, columns=["Title", "Min 22-day Return", "Max 22-day Return", "1st Quartile", "Latest 1-day Return", "Latest 22-day Return"])
csv_data = final_df.to_csv(index=False, header=True, sep=";").encode('utf-8')
print(final_df.head())

                                               Title  Min 22-day Return  \
0  1006.N - Goldman Sachs Parasol FIO Globalnej D...          -0.040077   
1       1007.N - MetLife PPE Fundusz Akcji B - Stooq          -0.016511   
2                1008.N - PKO Emerytura 2065 - Stooq          -0.022240   
3         1009.N - PKO Obligacji Uniwersalny - Stooq           0.003936   
4  1006.N - Goldman Sachs Parasol FIO Globalnej D...          -0.040077   

   Max 22-day Return  1st Quartile  Latest 1-day Return  Latest 22-day Return  
0           0.042157     -0.014637            -0.006401             -0.040077  
1           0.116253      0.019590             0.004369              0.050254  
2           0.080422      0.005791             0.006084              0.005319  
3           0.007159      0.004869             0.000169              0.005518  
4           0.042157     -0.014637            -0.006401             -0.040077  


In [ ]:
creds = service_account.Credentials.from_service_account_file('/tmp/credentials.json', scopes=['https://www.googleapis.com/auth/drive'])

drive_service = build('drive', 'v3', credentials=creds)

# File details
file_name = "oceny.csv"

# Correctly get the folder ID (replace 'Dane' with the actual folder name)
folder_name = 'Dane'
query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
results = drive_service.files().list(q=query, spaces='drive', fields='files(id)').execute()
items = results.get('files', [])

if items:
    folder_id = items[0]['id']
else:
    print(f"❌ Folder '{folder_name}' not found in Google Drive.")
    exit()

# Now use the correct folder ID in the file search query
query = f"name='{file_name}' and '{folder_id}' in parents"
results = drive_service.files().list(q=query, spaces='drive', fields='files(id)').execute()
items = results.get('files', [])

if items:
    file_id = items[0]['id']
    # Update the existing file as a new version
    # The MediaIoBaseUpload class needs to be called directly, not as an attribute of drive_service.
    media = MediaIoBaseUpload(io.BytesIO(csv_data), mimetype='text/csv', resumable=True)
    updated_file = drive_service.files().update(fileId=file_id, media_body=media).execute()
    print(f"File '{file_name}' updated as a new version. File ID: {file_id}")
else:
    # If the file doesn't exist, create it
    file_metadata = {
        'name': file_name,
        'parents': [folder_id] 
    }
    # The MediaIoBaseUpload class needs to be called directly, not as an attribute of drive_service.
    media = MediaIoBaseUpload(io.BytesIO(csv_data), mimetype='text/csv', resumable=True)
    created_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = created_file.get('id')
    print(f"File '{file_name}' created. File ID: {file_id}")

print(f"Analysis complete. Extracted data from {len(all_results)} pages saved in 'oceny.csv'.")